**17. `tf.data` API in TensorFlow**

In [14]:
import tensorflow as tf
import numpy as np

# import plotting libraries
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
from tensorflow.keras.datasets import mnist

(X_train,y_train),(X_test,y_test) = mnist.load_data()
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

**Using tf.data API from tensor-slices**

1. Create a Dataset from (features,labels) using `tf.data.Dataset.from_tensor_slices((features,labels))`. 
2. If we need to map some pre-processing function, we can do it using `map()`, which maps pre-processing function row by row (one element at a time, can be parallelized to multiple elements at a time) during iteration. 
3. We can use `.cache()` to cache the data which is helpful in speeding up the process, but needs space in RAM. 
4. We can use `.shuffle()` to shuffle the data.
5. We can use `.batch()` to create data batches.
6. We can use `.prefetch()`, this enhances parallel efficiency.

For test_dataset we can do caching after batching.


In [47]:
BATCH_SIZE = 32
RANDOM_SEED = 14
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))

print(train_dataset)

# function to pre-process data
def pre_process(img,label):
    img = tf.expand_dims(img,axis=-1)
    return tf.cast(img,tf.float32)/255.0, label

# applying transformation
train_dataset = train_dataset.map(pre_process,num_parallel_calls = tf.data.AUTOTUNE)
test_dataset = test_dataset.map(pre_process, num_parallel_calls=tf.data.AUTOTUNE)

print(train_dataset)

# caching
train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

print(train_dataset)


# shuffling
train_dataset = train_dataset.shuffle(buffer_size=1000,seed = RANDOM_SEED)
### we don't want to shuffle the test dataset

print(train_dataset)

# batching
train_dataset = train_dataset.batch(batch_size=BATCH_SIZE,drop_remainder=True,num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size=BATCH_SIZE,drop_remainder=True,num_parallel_calls=tf.data.AUTOTUNE)

print(train_dataset)

# prefetching
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print(train_dataset)

<_TensorSliceDataset element_spec=(TensorSpec(shape=(28, 28), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>
<_ParallelMapDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>
<CacheDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>
<_ShuffleDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>
<_ParallelBatchDataset element_spec=(TensorSpec(shape=(32, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(32,), dtype=tf.uint8, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(32,), dtype=tf.uint8, name=None))>


In [35]:
# visualizing elements in dataset (before batching)
# for img,label in train_dataset:
#     plt.imshow(img,cmap='gray')
#     plt.title(f'Number: {label}')
#     plt.show()
#     break


for X_batch,y_batch in train_dataset:
    print(X_batch.shape,y_batch)
    break 


(32, 28, 28) tf.Tensor([8 2 7 8 2 5 9 7 9 1 1 6 7 5 3 5 4 1 2 6 3 8 1 6 8 2 0 6 4 3 7 5], shape=(32,), dtype=uint8)


In [62]:
a = tf.constant(np.arange(9).reshape((3,3)))
a = tf.expand_dims(a,axis = 0)
a = tf.expand_dims(a,axis = -1)
print(a.shape)
tf.keras.layers.GlobalAveragePooling2D()(a)

(1, 3, 3, 1)


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4.]], dtype=float32)>

In [75]:
# creating a model using the functional API

def create_cnn(name = 'Model'):
    inputs = tf.keras.Input(shape=(28,28,1))

    # creating simplest convolutional-pool network
    x = tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3), strides=(1,1), 
                               activation='relu', padding = 'same',name = 'conv1')(inputs)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2), name = 'pool1')(x)

    x = tf.keras.layers.Conv2D(filters = 8, kernel_size=(3,3), strides=(1,1), 
                               activation='relu', padding = 'same', name = 'conv2')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2), name = 'pool2')(x)
    
    # global-max-pool
    x = tf.keras.layers.GlobalMaxPooling2D()(x)

    # creating fully connected-layers
    x = tf.keras.layers.Dense(16, activation= 'relu')(x)
    outputs = tf.keras.layers.Dense(10,activation = 'softmax')(x)

    return tf.keras.models.Model(inputs = inputs,outputs = outputs,name = name)


In [83]:
model = create_cnn()
model.summary()
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
model.compile(loss = loss, optimizer = optimizer, metrics=['accuracy'])

Model: "Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 28, 28, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 14, 14, 8)      │         1,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 7, 7, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d_6          │ (None, 8)              │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 10)             │           170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,634 (6.38 KB)

 Trainable params: 1,634 (6.38 KB)

 Non-trainable params: 0 (0.00 B)

In [84]:
%%time
history = model.fit(train_dataset,
          epochs = 5,
          verbose = 1,
          validation_data = test_dataset,
          )

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.3901 - loss: 1.7208 - val_accuracy: 0.7377 - val_loss: 0.7728
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7593 - loss: 0.7327 - val_accuracy: 0.8140 - val_loss: 0.5761
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8152 - loss: 0.5886 - val_accuracy: 0.8476 - val_loss: 0.4902
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8439 - loss: 0.5018 - val_accuracy: 0.8662 - val_loss: 0.4356
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8615 - loss: 0.4501 - val_accuracy: 0.8698 - val_loss: 0.4204
CPU times: user 6min 26s, sys: 20.7 s, total: 6min 47s
Wall time: 1min 1s


In [78]:
model.evaluate(test_dataset)

312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8724 - loss: 0.4016


[0.35771411657333374, 0.8863180875778198]

**Creating custom training loop**

In [106]:
model1 = create_cnn('Model1')
# model1.summary()
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

epochs = 5

In [109]:
%%time
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

def train_step(X_batch, y_batch):
    with tf.GradientTape() as tape:
        y_pred = model1(X_batch, training=True)
        loss = loss_function(y_batch, y_pred)
    grads = tape.gradient(loss, model1.trainable_variables)
    optimizer.apply_gradients(zip(grads, model1.trainable_variables))
    return loss

for epochi in range(epochs):
    batch_loss = []

    for X_batch, y_batch in train_dataset:
        loss = train_step(X_batch, y_batch)
        batch_loss.append(loss)

    mean_batch_loss = tf.reduce_mean(batch_loss)

    val_accuracy.reset_state()  # Clear for new epoch
    for X_val, y_val in test_dataset:
        val_accuracy.update_state(y_val, model1(X_val, training=False))

    accuracy = val_accuracy.result() * 100
    print(f'Epoch: {epochi+1}, Loss: {mean_batch_loss:.4f}, Accuracy: {accuracy:.2f} %')


Epoch: 1, Loss: 0.3910, Accuracy: 88.29 %
Epoch: 2, Loss: 0.3720, Accuracy: 89.46 %


2025-05-27 13:59:15.801907: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 3, Loss: 0.3575, Accuracy: 87.39 %
Epoch: 4, Loss: 0.3453, Accuracy: 89.84 %
Epoch: 5, Loss: 0.3357, Accuracy: 90.07 %
CPU times: user 10min 43s, sys: 1min 39s, total: 12min 22s
Wall time: 6min 4s


In [110]:
%%time
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

@tf.function
def train_step(X_batch, y_batch):
    with tf.GradientTape() as tape:
        y_pred = model1(X_batch, training=True)
        loss = loss_function(y_batch, y_pred)
    grads = tape.gradient(loss, model1.trainable_variables)
    optimizer.apply_gradients(zip(grads, model1.trainable_variables))
    return loss

for epochi in range(epochs):
    batch_loss = []

    for X_batch, y_batch in train_dataset:
        loss = train_step(X_batch, y_batch)
        batch_loss.append(loss)

    mean_batch_loss = tf.reduce_mean(batch_loss)

    val_accuracy.reset_state()  # Clear for new epoch
    for X_val, y_val in test_dataset:
        val_accuracy.update_state(y_val, model1(X_val, training=False))

    accuracy = val_accuracy.result() * 100
    print(f'Epoch: {epochi+1}, Loss: {mean_batch_loss:.4f}, Accuracy: {accuracy:.2f} %')


Cause: Unable to locate the source code of <function train_step at 0x7d0eadd0ab00>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch: 1, Loss: 0.3278, Accuracy: 90.05 %
Epoch: 2, Loss: 0.3208, Accuracy: 90.10 %
Epoch: 3, Loss: 0.3139, Accuracy: 90.58 %
Epoch: 4, Loss: 0.3093, Accuracy: 90.61 %
Epoch: 5, Loss: 0.3034, Accuracy: 89.78 %
CPU times: user 6min 41s, sys: 19.1 s, total: 7min
Wall time: 1min 7s
